In [129]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RepeatedKFold

In [133]:
train_df = pd.read_csv('../../feature_data/train/featureInput_noChampionRole_ZSCORED.csv')
# train_df = pd.read_csv('train_update.csv')
test_df = pd.read_csv('../../feature_data/test/featureInput_noChampionRole_ZSCORED.csv')
# test_df = pd.read_csv('test_update.csv')

In [135]:
train_df.head(3)

,bResult,btPlayerRole,bjPlayerRole,bmPlayerRole,baPlayerRole,bsPlayerRole,rtPlayerRole,rjPlayerRole,rmPlayerRole,raPlayerRole,...,raPlayerChampion,rsPlayerChampion,bCoopPlayer,rCoopPlayer,vsPlayer,bCoopChampion,rCoopChampion,vsChampion,bTeamColor,rTeamColor
0,1,0.252250,-0.113933,1.010727,0.434609,0.245249,0.160813,0.366454,0.119546,0.858685,...,0.133663,-1.850269,0.366567,0.175634,0.127328,-0.187979,1.213948,-0.206860,1.103748,0.920580
1,0,0.107836,-0.258477,0.248977,-0.715370,-0.487638,0.034632,0.263361,-0.826679,0.262123,...,0.179959,0.263562,-0.418938,-0.767054,-1.588838,-1.739330,0.131607,-1.303218,-0.459587,-0.379011
2,1,-0.363338,0.955698,0.776230,-0.188098,1.248147,1.081633,0.469547,0.290217,0.775167,...,0.545983,-1.850269,-0.093366,-0.831687,1.020567,-0.054209,-0.383016,1.589459,0.383076,1.106054


In [139]:
train_df.columns

Index(['bResult', 'btPlayerRole', 'bjPlayerRole', 'bmPlayerRole',
       'baPlayerRole', 'bsPlayerRole', 'rtPlayerRole', 'rjPlayerRole',
       'rmPlayerRole', 'raPlayerRole', 'rsPlayerRole', 'btPlayerChampion',
       'bjPlayerChampion', 'bmPlayerChampion', 'baPlayerChampion',
       'bsPlayerChampion', 'rtPlayerChampion', 'rjPlayerChampion',
       'rmPlayerChampion', 'raPlayerChampion', 'rsPlayerChampion',
       'bCoopPlayer', 'rCoopPlayer', 'vsPlayer', 'bCoopChampion',
       'rCoopChampion', 'vsChampion', 'bTeamColor', 'rTeamColor'],
      dtype='object')

In [71]:
# drop_cols = list(train_df.filter(regex='ChampionRole$').columns)
drop_cols = ['Unnamed: 0']

In [73]:
train_df = train_df.drop(drop_cols, axis=1)
train_df.columns

Index(['bResult', 'btPlayerRole', 'bjPlayerRole', 'bmPlayerRole',
       'baPlayerRole', 'bsPlayerRole', 'rtPlayerRole', 'rjPlayerRole',
       'rmPlayerRole', 'raPlayerRole', 'rsPlayerRole', 'btPlayerChampion',
       'bjPlayerChampion', 'bmPlayerChampion', 'baPlayerChampion',
       'bsPlayerChampion', 'rtPlayerChampion', 'rjPlayerChampion',
       'rmPlayerChampion', 'raPlayerChampion', 'rsPlayerChampion',
       'bCoopPlayer', 'rCoopPlayer', 'bCoopChampion', 'rCoopChampion',
       'vsPlayer', 'vsChampion', 'bTeamColor', 'rTeamColor'],
      dtype='object')

In [141]:
len(list(train_df.columns))

29

In [143]:
test_df = test_df.drop(drop_cols, axis=1)
len(list(test_df.columns))

KeyError: "['Unnamed: 0'] not found in axis"

In [145]:
y_train = train_df.loc[:, 'bResult']
X_train = train_df.loc[:, train_df.columns != 'bResult']
y_test = test_df.loc[:, 'bResult']
X_test = test_df.loc[:, test_df.columns != 'bResult']

In [147]:
X_train.count

<bound method DataFrame.count of       btPlayerRole  bjPlayerRole  bmPlayerRole  baPlayerRole  bsPlayerRole  \
0         0.252250     -0.113933      1.010727      0.434609      0.245249   
1         0.107836     -0.258477      0.248977     -0.715370     -0.487638   
2        -0.363338      0.955698      0.776230     -0.188098      1.248147   
3         0.178569      0.045067     -0.025138     -0.175134      0.548104   
4         2.831061      0.661353      0.758049      1.205383      0.490318   
...            ...           ...           ...           ...           ...   
6818      0.920830      0.753336     -1.787310     -0.131543     -0.857938   
6819     -0.263513     -1.270290     -1.278238     -3.327397     -0.055620   
6820     -1.650736     -0.308239     -3.314526     -3.327397     -1.760546   
6821     -0.156803     -0.258477      1.466323     -0.953493      0.345539   
6822     -3.358087     -3.293916     -2.805454     -2.816349     -2.061415   

      rtPlayerRole  rjPlayerRo

In [149]:
gnb = GaussianNB()

In [151]:
y_train.dtypes

dtype('int64')

In [153]:
gnb.fit(X_train, y_train)

GaussianNB()

In [155]:
# apparently the model must be fit prior to calling this
priors = gnb.predict_proba(X_test)
priors

array([[1.00000000e+00, 7.36494662e-18],
       [1.00000000e+00, 4.74563816e-17],
       [9.99999999e-01, 5.15996217e-10],
       ...,
       [1.00000000e+00, 6.84579632e-17],
       [1.00000000e+00, 3.61233865e-14],
       [8.06611879e-15, 1.00000000e+00]])

In [157]:
y_predict = gnb.predict(X_test)
mean_accuracy = gnb.score(X_test, y_test)
mean_accuracy

0.9960474308300395

In [159]:
accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.9960474308300395


In [161]:
len(priors)

759

In [163]:
# tenfold training
rkf = RepeatedKFold(n_repeats=5, n_splits=10, random_state=3)

In [165]:
import numpy as np

In [167]:
X_array = X_train.to_numpy()
y_array = y_train.to_numpy()

In [169]:
accuracies = []

In [171]:
best_probs = {'prob_blue': 0, 'prob_red': 0}

In [173]:
best_accuracy = 0

In [175]:
for train_index, test_index in rkf.split(X_train):
    # print(f'    Train: index={train_index}')
    # print(f'    Test: index={test_index}')
    X_fold_train, X_fold_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]
    prob_blue = len([x for x in list(y_fold_train) if x == 1]) / len(list(y_fold_train))
    prob_red = len([x for x in list(y_fold_train) if x == 0]) / len(list(y_fold_train))
    print(f'probability that blue wins: {prob_blue}')
    print(f'probability that red wins: {prob_red}')
    model = GaussianNB()
    model.fit(X_fold_train, y_fold_train)
    fold_prediction = model.predict(X_fold_test)
    accuracy = accuracy_score(y_fold_test, fold_prediction)
    accuracies.append(accuracy)
    print(f'accuracy: {accuracy}')
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_probs['prob_blue'] = prob_blue
        best_probs['prob_red'] = prob_red

probability that blue wins: 0.5441368078175896
probability that red wins: 0.45586319218241045
accuracy: 0.8740849194729137
probability that blue wins: 0.5438110749185667
probability that red wins: 0.4561889250814332
accuracy: 0.890190336749634
probability that blue wins: 0.5384364820846905
probability that red wins: 0.4615635179153095
accuracy: 0.9033674963396779
probability that blue wins: 0.542256961406937
probability that red wins: 0.45774303859306303
accuracy: 0.8973607038123167
probability that blue wins: 0.5443738804754926
probability that red wins: 0.45562611952450743
accuracy: 0.8914956011730205
probability that blue wins: 0.541117081908484
probability that red wins: 0.45888291809151605
accuracy: 0.907624633431085
probability that blue wins: 0.5412799218368344
probability that red wins: 0.4587200781631656
accuracy: 0.9002932551319648
probability that blue wins: 0.5440482006187918
probability that red wins: 0.4559517993812083
accuracy: 0.8973607038123167
probability that blue wi

In [177]:
accuracies

[0.8740849194729137,
 0.890190336749634,
 0.9033674963396779,
 0.8973607038123167,
 0.8914956011730205,
 0.907624633431085,
 0.9002932551319648,
 0.8973607038123167,
 0.9046920821114369,
 0.8797653958944281,
 0.9019033674963397,
 0.9077598828696926,
 0.8872620790629575,
 0.8856304985337243,
 0.9002932551319648,
 0.8826979472140762,
 0.8870967741935484,
 0.8929618768328446,
 0.9002932551319648,
 0.8929618768328446,
 0.8872620790629575,
 0.8945827232796486,
 0.890190336749634,
 0.8973607038123167,
 0.8900293255131965,
 0.8944281524926686,
 0.8958944281524927,
 0.8900293255131965,
 0.9017595307917888,
 0.9032258064516129,
 0.9033674963396779,
 0.8857979502196194,
 0.8799414348462665,
 0.907624633431085,
 0.8944281524926686,
 0.8841642228739003,
 0.8856304985337243,
 0.8973607038123167,
 0.9090909090909091,
 0.8914956011730205,
 0.8945827232796486,
 0.9136163982430454,
 0.9106881405563689,
 0.8988269794721407,
 0.8709677419354839,
 0.9017595307917888,
 0.8768328445747801,
 0.90615835777126

In [179]:
len(accuracies)

50

In [181]:
sum(accuracies) / 50

0.8943570499306572

In [183]:
best_accuracy

0.9136163982430454

In [185]:
best_probs

{'prob_blue': 0.5392508143322475, 'prob_red': 0.46074918566775247}

In [187]:
gnb = GaussianNB(priors=[best_probs['prob_red'], best_probs['prob_blue']])

In [189]:
gnb.fit(X_train, y_train)

GaussianNB(priors=[0.46074918566775247, 0.5392508143322475])

In [191]:
tuned_prediction = gnb.predict(X_test)

In [193]:
accuracy_w_priors = accuracy_score(y_test, tuned_prediction)
accuracy_w_priors

0.9960474308300395